# All PPTs

In [48]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [50]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 6.1 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [51]:
!pip install pyLDAvis==3.2.1

In [52]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [53]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [54]:
all = pd.read_csv('User Needs Assessments Data.csv')
all = all.loc[all['Person'] != 'Interviewer']

In [55]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [56]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [57]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [58]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [59]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [60]:
id2word[0]

'electricity'

In [61]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [62]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [63]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.107*"website" + 0.081*"issue" + 0.028*"service" + 0.028*"happen" + '
  '0.028*"governmental" + 0.028*"eventually" + 0.028*"leave" + '
  '0.028*"sometimes" + 0.028*"unfortunately" + 0.028*"inaubible"'),
 (1,
  '0.003*"trigger" + 0.003*"overall" + 0.003*"estimate" + 0.003*"viable" + '
  '0.003*"solution" + 0.003*"physical" + 0.003*"option" + 0.003*"check" + '
  '0.003*"interruption" + 0.003*"smooth"'),
 (2,
  '0.075*"think" + 0.046*"form" + 0.046*"month" + 0.031*"take" + '
  '0.031*"roughly" + 0.016*"customer" + 0.016*"send" + 0.016*"tell" + '
  '0.016*"account" + 0.016*"apply"'),
 (3,
  '0.034*"actually" + 0.034*"come" + 0.034*"call" + 0.034*"today" + '
  '0.034*"round" + 0.034*"know" + 0.034*"inform" + 0.018*"form" + '
  '0.018*"service" + 0.018*"obviously"'),
 (4,
  '0.045*"user" + 0.044*"take" + 0.043*"time" + 0.043*"clunky" + 0.043*"pay" + '
  '0.043*"friendly" + 0.043*"process" + 0.043*"hope" + 0.004*"website" + '
  '0.004*"people"'),
 (5,
  '0.037*"customer" + 0.037*"for

In [64]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.068423655807181

Coherence Score:  0.5069636103809895


In [65]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.119833  0.228038       1        1  20.985929
5      0.016105 -0.258835       2        1  13.400898
13    -0.289944 -0.063265       3        1   8.215471
18    -0.086016  0.226029       4        1   7.250024
16     0.162610 -0.221711       5        1   7.139415
17     0.270229 -0.079063       6        1   6.109071
2      0.031242  0.219981       7        1   5.506063
10    -0.076411 -0.210046       8        1   5.491281
12    -0.185860  0.198798       9        1   5.297874
11     0.194515  0.114833      10        1   5.026591
3      0.220369  0.021654      11        1   4.931694
14    -0.104685  0.090237      12        1   2.607198
0     -0.160680 -0.132714      13        1   2.474900
7     -0.179988 -0.011678      14        1   1.942935
8      0.105767 -0.060465      15        1   1.722861
4     -0.047437 -0.018327      16        1   0.955979
19     0.006614 -0.033834      17        1   0.402836
6     -0.004933  0.001879      18        1   0.297008
9      0.004807 -0.005683      19        1   0.122102
1      0.003864 -0.005828      20        1   0.119870, topic_info=        Term       Freq      Total Category  logprob  loglift
2        pay  17.000000  17.000000  Default  30.0000  30.0000
20      form  28.000000  28.000000  Default  29.0000  29.0000
166  website  10.000000  10.000000  Default  28.0000  28.0000
11   process   5.000000   5.000000  Default  27.0000  27.0000
71   provide   6.000000   6.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25       use   0.003462   7.326801  Topic20  -5.7462  -0.9309
26      care   0.003462   6.783922  Topic20  -5.7462  -0.8539
27       day   0.003462   5.279503  Topic20  -5.7462  -0.6032
28       end   0.003462   4.818139  Topic20  -5.7462  -0.5118
29     first   0.003462   3.967446  Topic20  -5.7462  -0.3175

[959 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
285       4  0.716325     able
193      14  0.884992   access
110       1  0.418937  account
110       3  0.139646  account
110       7  0.139646  account
...     ...       ...      ...
41        1  0.663552    whole
80        1  0.433554     work
80        5  0.433554     work
42        1  0.825802    wrong
296       4  0.716384     year

[603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 6, 14, 19, 17, 18, 3, 11, 13, 12, 4, 15, 1, 8, 9, 5, 20, 7, 10, 2])

In [66]:
pyLDAvis.save_html(vis, 'All PPTs.html')

# PPT01

In [22]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT01 = all.loc[(all['Person'] == 'PPT01') & (all['Person'] != 'Interviewer')]

In [23]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT01['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [24]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [25]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [28]:
id2word[0]

'electricity'

In [29]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [30]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [31]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"form" + 0.005*"take" + 0.005*"tell" + 0.005*"reply" + '
  '0.005*"service" + 0.005*"usually" + 0.005*"well" + 0.005*"print" + '
  '0.005*"fill" + 0.005*"send"'),
 (1,
  '0.077*"answer" + 0.077*"question" + 0.077*"satisfactory" + '
  '0.004*"available" + 0.004*"along" + 0.004*"bot" + 0.004*"chat" + '
  '0.004*"amount" + 0.004*"want" + 0.004*"actually"'),
 (2,
  '0.083*"time" + 0.082*"third" + 0.005*"customer" + 0.005*"ideally" + '
  '0.004*"form" + 0.004*"website" + 0.004*"people" + 0.004*"touchpoint" + '
  '0.004*"care" + 0.004*"chat"'),
 (3,
  '0.096*"provide" + 0.065*"authentication" + 0.065*"i" + 0.065*"login" + '
  '0.033*"datum" + 0.033*"safeguard" + 0.033*"factor" + 0.033*"d" + '
  '0.033*"ease" + 0.033*"website"'),
 (4,
  '0.041*"form" + 0.041*"take" + 0.040*"end" + 0.038*"month" + 0.038*"half" + '
  '0.021*"reply" + 0.021*"fill" + 0.021*"customer" + 0.021*"get" + '
  '0.021*"wrong"'),
 (5,
  '0.040*"people" + 0.040*"customer" + 0.030*"go" + 0.030*"website" + '
  

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.832702487707138

Coherence Score:  0.6421994319274502


In [67]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.119833  0.228038       1        1  20.985929
5      0.016105 -0.258835       2        1  13.400898
13    -0.289944 -0.063265       3        1   8.215471
18    -0.086016  0.226029       4        1   7.250024
16     0.162610 -0.221711       5        1   7.139415
17     0.270229 -0.079063       6        1   6.109071
2      0.031242  0.219981       7        1   5.506063
10    -0.076411 -0.210046       8        1   5.491281
12    -0.185860  0.198798       9        1   5.297873
11     0.194515  0.114833      10        1   5.026591
3      0.220369  0.021654      11        1   4.931694
14    -0.104685  0.090237      12        1   2.607198
0     -0.160680 -0.132714      13        1   2.474900
7     -0.179988 -0.011678      14        1   1.942935
8      0.105767 -0.060465      15        1   1.722861
4     -0.047437 -0.018327      16        1   0.955979
19     0.006614 -0.033834      17        1   0.402836
6     -0.004933  0.001879      18        1   0.297008
9      0.004807 -0.005683      19        1   0.122102
1      0.003864 -0.005828      20        1   0.119870, topic_info=        Term       Freq      Total Category  logprob  loglift
2        pay  17.000000  17.000000  Default  30.0000  30.0000
20      form  28.000000  28.000000  Default  29.0000  29.0000
166  website  10.000000  10.000000  Default  28.0000  28.0000
11   process   5.000000   5.000000  Default  27.0000  27.0000
71   provide   6.000000   6.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25       use   0.003462   7.326801  Topic20  -5.7462  -0.9309
26      care   0.003462   6.783922  Topic20  -5.7462  -0.8539
27       day   0.003462   5.279503  Topic20  -5.7462  -0.6032
28       end   0.003462   4.818139  Topic20  -5.7462  -0.5118
29     first   0.003462   3.967446  Topic20  -5.7462  -0.3175

[959 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
285       4  0.716325     able
193      14  0.884992   access
110       1  0.418937  account
110       3  0.139646  account
110       7  0.139646  account
...     ...       ...      ...
41        1  0.663552    whole
80        1  0.433554     work
80        5  0.433554     work
42        1  0.825802    wrong
296       4  0.716384     year

[603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 6, 14, 19, 17, 18, 3, 11, 13, 12, 4, 15, 1, 8, 9, 5, 20, 7, 10, 2])

In [68]:
pyLDAvis.save_html(vis, 'PPT01.html')

# PPT02

In [35]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT02 = all.loc[(all['Person'] == 'PPT02') & (all['Person'] != 'Interviewer')]

In [36]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT02['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']]


In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']


In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'bill', 'get', 'estimate', 'upcoming', 'bill', 'put', 'inaudible', 'meter', 'reading', 'get', 'estimate', 'monthly', 'bill', 'check', 'interruption', 'service']]


In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [41]:
id2word[0]

'bill'